# CAICT 風機結冰預測 - Data Resampling

2017/07/12  
http://www.industrial-bigdata.com/competition/competitionAction!showDetail.action?competition.competitionId=1

<a id='home'></a>
## Outline
[Load Data and Modules](#load)  
[read raw data](#read)  
[assign event id](#assign)    
[get event summary](#eventsummary)  
[data resampling](#resampling)
[save resample data to csv](#csv)

<a id='load'></a>
**Load Data and Modules**

- **load Python modules:**

In [1]:
%matplotlib inline
import numpy as np
import datetime
import pandas as pd
import os
import sys
from PIL import Image
import lzma
import pickle
from IPython.display import display
import shutil
import csv
from sklearn.metrics import precision_score
from sklearn.metrics import roc_curve, auc  
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns
from pylab import *
from IPython.core.interactiveshell import InteractiveShell
from operator import xor
#for resampling
#from imblearn.under_sampling import TomekLinks
#from imblearn.over_sampling import RandomOverSampler
#from imblearn.under_sampling import RandomUnderSampler
#from imblearn.over_sampling import SMOTE

import sklearn as sk
from sklearn.metrics import precision_score

pd.set_option('display.max_columns', None)
InteractiveShell.ast_node_interactivity = "all"


- ** data process functions **

In [29]:
def assign_eventid(data):
    label_col = data['label']
    label_col = label_col.values
    event_end=False
    event_id = 0;
    label_previous=0
    event_id_col = []
    for label in label_col: 
        if (label==0) & (label_previous==1):
            event_end=True
        label_previous = label;
        if event_end:
            event_end=False
            event_id=event_id+1
        event_id_col.append(event_id)    
    data['event']=event_id_col
    return data

def get_events_summary(data):
    events_info = pd.DataFrame(columns=('failStartTime','failEndTime','eventid', 'nor_min','unknown_min', 'abnor_min', 'nor_cnt', 'unknown_cnt', 'abnor_cnt','nor_idx_start','nor_idx_end','unknown_idx_start','abnor_idx_start', 'abnor_idx_end','no_unknown'))
    for eventid in set(data['event']):
        data_event = data[data['event']==eventid]
        evtdata_nor=data_event[data_event['label']==0]
        evtdata_abnor=data_event[data_event['label']==1]
        evtdata_unknown=data_event[data_event['label']==2]
    
        abnor_starttime = 0 if evtdata_abnor.empty else evtdata_abnor['time'].iloc[1]
        abnor_endtime = 0 if evtdata_abnor.empty else evtdata_abnor['time'].iloc[-1]
        
        nor_min = ((evtdata_nor['time'].iloc[-1] - evtdata_nor['time'].iloc[0]).total_seconds())/60
        abnor_min =  0 if evtdata_abnor.empty else ((evtdata_abnor['time'].iloc[-1] - evtdata_abnor['time'].iloc[0]).total_seconds())/60
        #unknown_min = ((evtdata_unknown['time'].iloc[-1] - evtdata_unknown['time'].iloc[0]).total_seconds())/60
        unknown_min = 0 if evtdata_unknown.empty else ((evtdata_unknown['time'].iloc[-1] - evtdata_unknown['time'].iloc[0]).total_seconds())/60    
        event_cnt = data_event['event'].count()
        nor_cnt = evtdata_nor['event'].count()
        abnor_cnt = 0 if evtdata_abnor.empty else evtdata_abnor['event'].count()
        #unknown_cnt= evtdata_unknown['event'].count()
        unknown_cnt = 0 if evtdata_unknown.empty else evtdata_unknown['event'].count()

        nor_idx_start= evtdata_nor.index[0]
        nor_idx_end= evtdata_nor.index[-1]
        
        abnor_idx_start= -1 if evtdata_abnor.empty else evtdata_abnor.index[0]
        abnor_idx_end= -1 if evtdata_abnor.empty else evtdata_abnor.index[-1]
        unknown_idx_start= abnor_idx_start if evtdata_unknown.empty else evtdata_unknown.index[0]

        no_known= 'Y' if evtdata_unknown.empty else 'N'
        events_info.loc[eventid] = [abnor_starttime, abnor_endtime, eventid, nor_min, unknown_min, abnor_min, nor_cnt, unknown_cnt, abnor_cnt,nor_idx_start,nor_idx_end, unknown_idx_start, abnor_idx_start, abnor_idx_end,no_known]
    return events_info


resample_policy_desc={
    '0': 'policy=0: no resample',
    '1': 'policy=1: event data + normal data (before and after event )',
    '2': 'policy=2: event data + normal data (before event only)'
}
%store resample_policy_desc

#根據事件的數據量, 從事件前後正常的區間取同樣多的數據
def resample_policy_p1(events_info):
    resample_info = pd.DataFrame(columns=('eventid', 'abnor_cnt','event_start', 'event_end'))
    for idx, row in events_info.iterrows():
        event_id = row['eventid']
        abnor_cnt = row['abnor_cnt']
        event_start = row['nor_idx_end'] - abnor_cnt
        event_end = row['abnor_idx_end'] + abnor_cnt
        if abnor_cnt==0:
            resample_info.loc[event_id] = [event_id, 0, 0, 0]
        else:
            resample_info.loc[event_id] = [event_id, abnor_cnt, event_start, event_end]
    return resample_info

#從不良事件的中間往前取pass:fail 2:1的數據
def resample_policy_p2(events_info):
    resample_info = pd.DataFrame(columns=('eventid', 'abnor_cnt','event_start', 'event_end'))
    for idx, row in events_info.iterrows():
        event_id = row['eventid']
        abnor_cnt = row['abnor_cnt']
        if abnor_cnt==0:
            resample_info.loc[event_id] = [event_id, 0, 0, 0]
            continue
        event_start = row['nor_idx_end']-abnor_cnt
        event_end = row['abnor_idx_start']+int(abnor_cnt/2)
        resample_info.loc[event_id] = [event_id, abnor_cnt, event_start, event_end]
    return resample_info

#根據事件的數據量, 從事件前正常的區間取同樣多的數據
def resample_policy_p3(events_info):
    resample_info = pd.DataFrame(columns=('eventid', 'abnor_cnt','event_start', 'event_end'))
    for idx, row in events_info.iterrows():
        event_id = row['eventid']
        abnor_cnt = row['abnor_cnt']
        event_start = row['nor_idx_end'] - abnor_cnt
        event_end = row['abnor_idx_end'] 
        if abnor_cnt==0:
            resample_info.loc[event_id] = [event_id, 0, 0, 0]
        else:
            resample_info.loc[event_id] = [event_id, abnor_cnt, event_start, event_end]
    return resample_info


def get_resample_data(data, policyfun, evt_sumy):
    resample_info = policyfun(evt_sumy)
    resample_data = pd.DataFrame()
    for idx, row in resample_info.iterrows():
        event_id = row['eventid']
        abnor_cnt = row['abnor_cnt']
        event_start = row['event_start']
        event_end = row['event_end']
        resample_data_i = data[event_start:event_end].copy()
        resample_data_i['event']=event_id
        resample_data =resample_data.append(resample_data_i)
    return resample_data

Stored 'resample_policy_desc' (dict)



***

<a id='read'></a>
**read raw data** 

In [53]:
data_M15 = pd.read_csv('data/goodformat_15.csv')
data_M21 = pd.read_csv('data/goodformat_21.csv')
data_M08 = pd.read_csv('data/ice1/test/08/08_data.csv')


NameError: name 'data_15' is not defined

<a id='change'></a>
** change time format from string to datetime**

In [6]:
data_M15['time']=pd.to_datetime(data_M15['time'])
data_M21['time']=pd.to_datetime(data_M21['time'])
data_M08['time']=pd.to_datetime(data_M08['time'])

data_M15['timestamp'] = data_M15['time'].apply(lambda x: x.timestamp())
data_M21['timestamp'] = data_M21['time'].apply(lambda x: x.timestamp())
data_M08['timestamp'] = data_M08['time'].apply(lambda x: x.timestamp())

<a id='assign'></a>
** assign event id**

In [7]:
data_M15 = assign_eventid(data_M15)
data_M21 = assign_eventid(data_M21)

<a id='eventsummary'></a>
** get event summary **

In [21]:
events_summary_M15 = get_events_summary(data_M15)
events_summary_M21 = get_events_summary(data_M21)
events_summary_M15
events_summary_M21

,failStartTime,failEndTime,eventid,nor_min,unknown_min,abnor_min,nor_cnt,unknown_cnt,abnor_cnt,nor_idx_start,nor_idx_end,unknown_idx_start,abnor_idx_start,abnor_idx_end,no_unknown
0,2015-11-04 21:37:00,2015-11-04 22:29:00,0,4206.0,0,52.0,5727,0,443,0,5840,6587,6587,7029,Y
1,2015-11-09 04:43:00,2015-11-09 06:35:00,1,5258.0,0,112.0,43020,0,265,7030,50049,50422,50422,50686,Y
2,2015-11-09 21:21:00,2015-11-09 23:14:00,2,413.0,0,113.0,3383,0,951,50687,54069,55070,55070,56020,Y
3,2015-11-16 03:52:00,2015-11-16 16:08:00,3,8579.0,0,737.0,39486,0,2101,56021,95506,95621,95621,98721,Y
4,2015-11-21 10:07:00,2015-11-21 10:20:00,4,4978.0,0,13.0,35032,0,106,98722,133753,133893,133893,133998,Y
5,2015-11-23 06:58:00,2015-11-23 07:39:00,5,2140.0,0,41.0,14016,0,352,133999,148014,148245,148245,148596,Y
6,2015-11-23 18:46:00,2015-11-26 01:40:00,6,297.0,0,3294.0,2120,0,3188,148597,150716,151717,151717,156253,Y
7,2015-11-29 02:04:00,2015-11-29 03:57:00,7,3799.0,0,113.0,29550,0,951,156254,185803,186804,186804,187754,Y
8,2015-11-29 17:30:00,2015-11-30 06:50:00,8,154.0,0,800.0,1321,0,1254,187755,189075,190107,190107,191360,Y
9,2015-11-30 23:22:00,2015-11-30 23:39:00,9,399.0,0,17.0,3212,0,148,191361,194572,194573,194573,194720,Y


,failStartTime,failEndTime,eventid,nor_min,unknown_min,abnor_min,nor_cnt,unknown_cnt,abnor_cnt,nor_idx_start,nor_idx_end,unknown_idx_start,abnor_idx_start,abnor_idx_end,no_unknown
0,2015-11-04 22:15:00,2015-11-04 23:33:00,0,4449.0,0,78.0,12020,0,659,0,12019,12290,12290,12948,Y
1,2015-11-09 03:22:00,2015-11-09 05:14:00,1,5177.0,0,113.0,40083,0,747,12949,53031,54032,54032,54778,Y
2,2015-11-09 21:26:00,2015-11-09 23:18:00,2,427.0,0,112.0,2437,0,951,54779,57215,58216,58216,59166,Y
3,2015-11-13 02:59:00,2015-11-13 04:51:00,3,3595.0,0,112.0,20717,0,951,59167,79883,80459,80459,81409,Y
4,2015-11-16 15:31:00,2015-11-16 15:57:00,4,3325.0,0,26.0,9716,0,217,81410,91125,91126,91126,91342,Y
5,2015-11-23 20:40:00,2015-11-23 22:33:00,5,9027.0,0,113.0,48850,0,434,91343,140192,141193,141193,141626,Y
6,2015-11-24 05:42:00,2015-11-24 06:31:00,6,75.0,0,49.0,637,0,414,141627,142263,143055,143055,143468,Y
7,2015-11-24 14:59:00,2015-11-24 16:51:00,7,83.0,0,113.0,477,0,951,143469,143945,144946,144946,145896,Y
8,2015-11-25 20:55:00,2015-11-26 03:40:00,8,56.0,0,405.0,472,0,1407,145897,146368,147369,147369,148997,Y
9,2015-11-28 04:15:00,2015-11-28 06:08:00,9,2406.0,0,113.0,17021,0,951,148998,166018,167019,167019,167969,Y


<a id='resampling'></a>
** data resampling **

- ** policy 1 **

In [51]:
resample_data_p1_M15_ = get_resample_data(data_M15, resample_policy_p1, events_summary_M15)
resample_data_p1_M21_ = get_resample_data(data_M21, resample_policy_p1, events_summary_M21)

- ** policy 2**

In [54]:
resample_data_p2_M15 = get_resample_data(data_M15, resample_policy_p2, events_summary_M15)
resample_data_p2_M21 = get_resample_data(data_M21, resample_policy_p2, events_summary_M21)

- ** policy 3**


In [32]:
resample_data_p3_M15 = get_resample_data(data_M15, resample_policy_p3, events_summary_M15)
resample_data_p3_M21 = get_resample_data(data_M21, resample_policy_p3, events_summary_M21)

<a id='csv'></a>
** save resample data to csv**

In [ ]:
resample_data_p0_M15 = data_M15
resample_data_p0_M21 = data_M21
resample_data_p0_M15.to_csv('data/resample_data_p0_15.csv',index=False)
resample_data_p1_M15.to_csv('data/resample_data_p1_15.csv',index=False)
resample_data_p2_M15.to_csv('data/resample_data_p2_15.csv',index=False)
resample_data_p3_M15.to_csv('data/resample_data_p3_15.csv',index=False)
resample_data_p0_M21.to_csv('data/resample_data_p0_21.csv',index=False)
resample_data_p1_M21.to_csv('data/resample_data_p1_21.csv',index=False)
resample_data_p2_M21.to_csv('data/resample_data_p2_21.csv',index=False)
resample_data_p3_M21.to_csv('data/resample_data_p3_21.csv',index=False)

In [55]:
resample_data_p3_M15.to_csv('data/resample_data_p3_15.csv',index=False)
resample_data_p3_M21.to_csv('data/resample_data_p3_21.csv',index=False)

[home](#home)

***